In [5]:
import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
from math import sqrt
from numba import njit,jit

In [6]:
@njit
def mydist(XA,XB,mode= 'euclidean'):
    if mode == 'euclidean':
        aux3 = []
        for i in range (len(XA)):
            aux2 = []
            for j in range (len(XB)):
                aux1 = []
                bux1 = 0 
                # calculate distance
                for e1, e2 in zip(XB[j],XA[i,:]):
                    aux1.append((e1-e2)**2)
                    bux1 += ((e1-e2)**2)

                    #aux1.append(sqrt(sum((e1-e2)**2)))
                aux2.append(bux1**(0.5))
            aux3.append(aux2)
        return aux3
    
    if mode == 'cosine':
        aux2 = []
        for i in range(len(XA)):
            aux1 = []
            for j in range(len(XB)):
                A = Uniquesample[j,:] - 1
                B = Uniquesample[i,]
                dot = 0
                denom_a = 0
                denom_b = 0
                for k in range (len(XB[j,:])):
                    dot += (XB[j,k]*XA[i,k])
                    denom_a += (XB[j,k] * XB[j,k])
                    denom_b += (XA[i,k] * XA[i,k])

                aux1.append(1 - ((dot / ((denom_a ** 0.5) * (denom_b ** 0.5)))))
            aux2.append(aux1)

        return aux2

In [28]:
#@njit
def chessboard_division(Uniquesample, MMtypicality, interval1, interval2, distancetype):
    L, W = Uniquesample.shape
    if distancetype == 'euclidean':
        W = 1
    BOX = [Uniquesample[k] for k in range(W)]
    BOX_miu = [Uniquesample[k] for k in range(W)]
    BOX_S = [1]*W
    BOX_X = [np.sum(Uniquesample[k]**2) for k in range(W)]
    NB = W
    BOXMT = [MMtypicality[k] for k in range(W)]

    for i in range(W,L):
        XA = Uniquesample[i].reshape(1,-1)
        XB = BOX_miu
        a = []
        for i in range (len(XA)):
            aux2 = []
            for j in range (len(XB)):
                aux1 = []
                bux1 = 0 
                # calculate distance
                for e1, e2 in zip(XB[j],XA[i,:]):
                    aux1.append((e1-e2)**2)
                    bux1 += ((e1-e2)**2)

                    #aux1.append(sqrt(sum((e1-e2)**2)))
                aux2.append(bux1**(0.5))
            a.append(aux2)
            
        b = np.sqrt(mydist(Uniquesample[i].reshape(1,-1), BOX_miu, mode='cosine'))
        
        aux3 = []
        for i in range(len(XA)):
            aux1 = []
            for j in range(len(XB)):
                A = Uniquesample[j,:] - 1
                B = Uniquesample[i,]
                dot = 0
                denom_a = 0
                denom_b = 0
                for k in range (len(XB[j])):
                    dot += (XB[j,k]*XA[i,k])
                    denom_a += (XB[j,k] * XB[j,k])
                    denom_b += (XA[i,k] * XA[i,k])

                aux1.append(1 - ((dot / ((denom_a ** 0.5) * (denom_b ** 0.5)))))
            b.append(aux1)
        
        distance = np.array([a,b]).T.reshape(1,-1)
        
        print(distance.shape)

        SQ = []
        for j,d in enumerate(distance):
            if d[0] < interval1 and d[1] < interval2:
                SQ.append(j)
                
        COUNT = len(SQ)
        if COUNT == 0:
            BOX.append(Uniquesample[i])
            NB = NB + 1
            BOX_S.append(1)
            BOX_miu.append(Uniquesample[i])
            BOX_X.append(sum(Uniquesample[i]**2))
            BOXMT.append(MMtypicality[i])
        if COUNT >= 1:
            DIS = distance[SQ[::],0]/interval1 + distance[SQ[::],1]/interval2 # pylint: disable=E1136  # pylint/issues/3139
            b = np.argmin(DIS)
            BOX_S[SQ[b]] = BOX_S[SQ[b]] + 1
            BOX_miu[SQ[b]] = (BOX_S[SQ[b]]-1)/BOX_S[SQ[b]]*BOX_miu[SQ[b]] + Uniquesample[i]/BOX_S[SQ[b]]
            BOX_X[SQ[b]] = (BOX_S[SQ[b]]-1)/BOX_S[SQ[b]]*BOX_X[SQ[b]] + sum(Uniquesample[i]**2)/BOX_S[SQ[b]]
            BOXMT[SQ[b]] = BOXMT[SQ[b]] + MMtypicality[i]

In [29]:
Uniquesample = np.array(([1,2,3],[1,5,7],[3,8,9]))
MMtypicality = np.array(([43,29,30]))
distancetype = 'euclidean'
interval1 = np.array([6.86690614])
interval2 = np.array([0.99159042])


In [30]:
%%time
chessboard_division(Uniquesample, MMtypicality, interval1, interval2, distancetype)

TypeError: list indices must be integers or slices, not tuple